In [2]:
import pandas as pd
import cudf
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GPT2LMHeadModel, GPT2Tokenizer
import time

# Filtrando os dados Originais

In [2]:
tweets1 = pd.read_excel('tweets1.xlsx')
tweets2 = pd.read_excel('tweets2.xlsx')

In [3]:
tweets_combined = pd.concat([tweets1, tweets2])

In [10]:
tweets_english = tweets_combined[tweets_combined['Tweet Language'] == 'English'].copy()

In [12]:
tweets_english['ID'] = range(1, len(tweets_english) + 1)

In [15]:
tweets_filtered = tweets_english[['ID', 'Tweet Content']].copy()

In [16]:
tweets_filtered.head()

,ID,Tweet Content
1,1,"""𝐈𝐭. 𝐂𝐚𝐧𝐧𝐨𝐭. 𝐆𝐞𝐭. 𝐁𝐢𝐠𝐠𝐞𝐫. 𝐓𝐡𝐚𝐧. 𝐓𝐡𝐢𝐬. 🔥\n\nGet..."
3,2,"""It’ll be a tough night for Europe today.\n\n#..."
4,3,"""In defeat or in victory, always say Alhamduli..."
5,4,"""FAFC Genesis Edition ( This collection have m..."
6,5,"""Get ready for zabardast action on #25th Jan ...."


In [17]:
tweets_filtered.to_csv('tweets_filtered.csv', index=False)

# Carregando dados na GPU

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

In [4]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [6]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", 
                                             torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
model.resize_token_embeddings(len(tokenizer))

Embedding(32001, 4096)

In [8]:
model.to("cuda")

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32001, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [27]:
# Ler dados do CSV usando cudf
df = cudf.read_csv('tweets_filtered.csv')

# Converter a coluna de tweets para uma lista no cudf (mantendo na GPU)
tweets = df['Tweet Content'].head(10).to_arrow().to_pylist()


In [28]:
len(tweets)

10

In [64]:
def classify_tweets(tweets, batch_size=3):
    results = []
    prompt = "Classify the following tweets as to whether it mentions sports betting, answer with yes or no, one per line: "
    
    for i in range(0, len(tweets), batch_size):
        batch = tweets[i:i + batch_size]
        str_for_promp = ''
        for k in range(len(batch)):
            str_for_promp += batch[k] + '\n'
        batch_with_prompt = prompt + '\n' + str_for_promp
        print(batch_with_prompt)
        inputs = tokenizer(batch_with_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
        # Usar autocast para mixed precision
        with torch.cuda.amp.autocast():
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=20)  # Ajuste max_new_tokens conforme necessário
        
        results.extend([tokenizer.decode(output, skip_special_tokens=True) for output in outputs])
    
    return results

In [65]:
start_time = time.time()
results = classify_tweets(tweets)
end_time = time.time()

Classify the following tweets as to whether it mentions sports betting, answer with yes or no, one per line: 
"𝐈𝐭. 𝐂𝐚𝐧𝐧𝐨𝐭. 𝐆𝐞𝐭. 𝐁𝐢𝐠𝐠𝐞𝐫. 𝐓𝐡𝐚𝐧. 𝐓𝐡𝐢𝐬. 🔥

Get into the #FIFAWorldCup Final mode with none other than @iamsrk &amp; @WayneRooney on Dec 18, LIVE on #JioCinema &amp; #Sports18 📺📲

#Qatar2022 #ARGFRA #WorldsGreatestShow #FIFAWConJioCinema #FIFAWConSports18 #Pathaan"
"It’ll be a tough night for Europe today.

#FranceVsMorocco @ #FIFAWorldCup.

Islamîsts will either celebrate the win of Islàm or avenge the defeat of Islàm."
"In defeat or in victory, always say Alhamdulillah and keep thanking Allah. And surely, Allah is the best of planners 🇲🇦♥️ #FIFAWorldCup https://t.co/CLSicv9g0h"

Classify the following tweets as to whether it mentions sports betting, answer with yes or no, one per line: 
"FAFC Genesis Edition ( This collection have maximum supply of 107 NFTs) : https://t.co/bjvgIJMyRm

Full collection of 7,910 NFTs will be launched on 16’Dec’2022! ⚽️🥅

Join Discord : https://t.co

In [66]:
results[2]

'Classify the following tweets as to whether it mentions sports betting, answer with yes or no, one per line: \n"Morocco 🇲🇦 can be PROUD of their World Cup #FIFAWorldCupQatar2022 👏👏👏\n\nThe ATLAS LIONS 🇲🇦 took down Belgium, Spain and Portugal on their way to becoming the FIRST AFRICAN NATION to ever make the #FIFAWorldCup semifinal. ❤️👏\n\nAn incredible show of resilience and hard work. 👊 https://t.co/ARE2AKF6XZ"\n"Name the religion behind this rioting in France 🇫🇷 post their victory in #FIFAWorldCup against #Morocco https://t.co/vZ2XdE6Emo"\n"“This crazy fanbase leaves everything for the Cup!” 🎶 #ARG\n\n#Qatar2022 | #FIFAWorldCup https://t.co/3yWdNFM6Lp"\n"Lionel Messi has been named the 2022 #FIFAWorldC'

In [ ]:
print(f"Tempo total: {end_time - start_time:.2f} segundos")

In [5]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
model.to("cuda")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)